# Implementación de modelo para la detección de luces en semáforos peatonales

## Preparación de los archivos

Se convierte las anotaciones de objetos desde un formato XML al formato de texto utilizado por YOLO.

In [ ]:
import os
import xml.etree.ElementTree as ET

def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_annotation(xml_file, output_dir, classes):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    with open(os.path.join(output_dir, os.path.splitext(os.path.basename(xml_file))[0] + '.txt'), 'w') as out_file:
        for obj in root.iter('object'):
            cls = obj.find('name').text
            if cls not in classes:
                continue
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
                 float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            bb = convert((w, h), b)
            out_file.write(f"{cls_id} " + " ".join([str(a) for a in bb]) + '\n')

# Define the classes
classes = ["red light", "green light"]

# Paths
current_path = os.getcwd()
xml_dir = os.path.join(current_path,'dataset/annotations/xml')
output_dir = os.path.join(current_path,'dataset/annotations/output')
os.makedirs(output_dir, exist_ok=True)

# Convert all XML files
for xml_file in os.listdir(xml_dir):
    if xml_file.endswith('.xml'):
        convert_annotation(os.path.join(xml_dir, xml_file), output_dir, classes)

## Entrenamiento

 Se realiza un proceso de fine-tuning de un modelo YOLOv8m pre-entrenado para adaptarlo a la tarea de detección de luces de semáforos peatonales. Utiliza un conjunto de datos personalizado definido en 'TrafficLight.yaml'. El modelo resultante, optimizado para la detección de semáforos, se guarda para su uso posterior.

In [ ]:
from ultralytics import YOLO
from multiprocessing import freeze_support
import os

# Cargar el modelo pre-entrenado YOLOv8m
model_path = os.path.join(os.path.dirname(__file__), 'yolov8m.pt')
model = YOLO(model_path)
current_path = os.getcwd()

# Realizar fine-tuning
if __name__ == '__main__':
    freeze_support()
    results = model.train(
        data=os.path.join(current_path,'TrafficLight.yaml'),
        epochs=80,
        imgsz=640,
        batch=16,
        name='yolov8m_traffic_light',
        patience=20,
        device=0
    )

    # Guardar el modelo entrenado
    model.save('yolov8m_TrafficLight.pt')

## Testing

In [ ]:
from ultralytics import YOLO
import os

path = os.path.join(os.path.dirname(__file__), 'best.pt')
model=YOLO(path)

results = model.predict(source='dataset/images/test/noche/',save = True, save_txt=True)